In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from collections import ChainMap
from contextlib import contextmanager
from functools import wraps, update_wrapper
import inspect
from inspect import Parameter, getsource, getsourcefile, getfile, ismodule, \
    ismethod, isfunction

from htools import *

In [3]:
@contextmanager
def temporary_global_scope(kwargs):
    # Make kwargs temporarily available as global vars.
    old_globals = globals().copy()
    globals().update(kwargs)
    try:
        yield
    finally:
        for k in kwargs:
            if k in old_globals:
                globals()[k] = old_globals[k]
            else:
                del globals()[k]

In [4]:
k = 1
print('\nPRE:')
print('k:', k)
try: 
    del v
except:
    pass
with assert_raises(NameError):
    print('v:', v)
    
with temporary_global_scope({'z': 3, 'k': 4}):
    print('\nIN:')
    print('k:', k)
    print('z:', z)
    
print('\nPOST:')
print('k:', k)
with assert_raises(NameError):
    print('z:', z)


PRE:
k: 1
As expected, got NameError(name 'v' is not defined).

IN:
k: 4
z: 3

POST:
k: 1
As expected, got NameError(name 'z' is not defined).


In [41]:
"""Realized this updates function signature (so the user sees the desired 
arguments when hitting shift-tab) but it doesn't actually update the args
accepted by the function. Options:
1. Construct a new function with the desired params. This is more robust
since it should ensure the correct values of __defaults__, __kwedefaults__,
etc.

UPDATE: Proposed method 2 (simply adding args/kwargs) does not work since
vars will be available in the decorated func as kwargs['a'] rather than a. At
the moment, I've implemented a seemingly working hack to make them available
as temporary global kwargs. Would prefer to get the signature to match the 
real function but this is something. At the moment, we don't support adding
args or kwargs from *fns (I'm thinking kwargs might be possible in some way by
prepending them with the func name, e.g. foo_kwargs, tri_kwargs, etc. Have to
think about if this would work for args.)

Should also think about if this is really the desired interface. Might be hard
to remember what variables are available because they're not explicitly shown
in code, even if they are in the signature. Realized fastai's delegates does
something a bit different: func still accepts kwargs, you just see the names
in the docstring. This is probably much safer, though the use case is a little
different.
"""
def add_kwargs(*fns, required=True, variable=True):
    """When one or more functions are called inside another function, we often
    have the choice of accepting **kwargs in our outer function (downside: 
    user can't see parameter names with quick documentation tools) or 
    explicitly typing out each parameter name and default (downsides: time 
    consuming and error prone since it's easy to update the inner function and
    forget to update the outer one). This lets us update the outer function's
    signature automatically based on the inner function(s)'s signature(s).
    The Examples section should make this more clear.
    
    The wrapped function must accept **kwargs, but you shouldn't refer to 
    `kwargs` explicitly inside the function. Its variables will be made
    available essentially as global variables.
    
    Note: don't actually use this for anything important, I imagine it could
    lead to some pretty nasty bugs. I was just determined to get something
    working.
    
    Parameters
    ----------
    fns: functions
        The inner functions whose signatures you wish to use to update the
        signature of the decorated outer function. When multiple functions 
        contain a parameter with the same name, priority is determined by the
        order of `fns` (earlier means higher priority).
    required: bool
        If True, include required arguments from inner functions (that is,
        positional arguments or positional_or_keyword arguments with no 
        default value). If False, exclude these (it may be preferable to 
        explicitly include them in the wrapped function's signature).
    variable: bool
        If True, include *kwargs and **kwargs from the inner functions. They
        will be made available as {inner_function_name}_args and 
        {inner_function_name}_kwargs, respectively (see Examples). Otherwise,
        they will be excluded.
    
    Examples
    --------
    def foo(x, c, *args, a=3, e=(11, 9), b=True, f=('a', 'b', 'c'), **kwargs):
        print('in foo')
        return x * c
        
    def baz(n, z='z', x='xbaz', c='cbaz'):
        print('in baz')
        return n + z + x + c
        
    baz comes before foo so its x param takes priority and has a default
    value of 'xbaz'. The decorated function always retains first priority so
    the c param remains positional despite its appearance as a positional
    arg in foo.
    
    @add_kwargs(baz, foo, positional=True)
    def bar(c, d=16, **kwargs):
        foo_res = foo(x, c, *foo_args, a=a, e=e, b=b, f=f, **foo_kwargs)
        baz_res = baz(n, z, x, c)
        return {'c': c, 'n': n, 'd': d, 'x': x, 'z': z, 'a': a, 
                'e': e, 'b': b, 'f': f}
                
    bar ends up with the following signature:
    <Signature (c, n, d=16, x='xtri', foo_args=(), z='z', *, a=3, e=(11, 9),
                b=True, f=('a', 'b', 'c'), foo_kwargs={}, **kwargs)>
                
    Notice many variables are available inside the function even though they
    aren't explicitly hard-coded into our function definition. When using
    shift-tab in Jupyter or other quick doc tools, they will all be visible.
    You can see how passing in multiple functions can quickly get messy so
    if you insist on using this, try to keep it to 1-2 functions if possible.
    """
    param_types = {Parameter.POSITIONAL_OR_KEYWORD, Parameter.KEYWORD_ONLY}
    if required: param_types.add(Parameter.POSITIONAL_ONLY)
        
    def _args(fn): 
        res = {}
        for k, v in params(fn).items():
            # If required=False, allow positional_or_keyword args with
            # defaults but not those without. 
            if v.kind in param_types and (required 
                                          or v.default != inspect._empty):
                res[k] = v
                
            # args/kwargs are converted to non-varying types and names are
            # adjusted to include function name. E.g. if we're adding kwargs
            # from function foo which accepts kwargs, that arg becomes a 
            # keyword-only dictionary called foo_kwargs.
            elif variable:
                name = f'{fn.__name__}_{k}'
                if v.kind == Parameter.VAR_POSITIONAL:
                    kind = Parameter.POSITIONAL_OR_KEYWORD
                    default = ()
                elif v.kind == Parameter.VAR_KEYWORD:
                    kind = Parameter.KEYWORD_ONLY
                    default = {}
                else:
                    continue
                res[name] = Parameter(name, kind, default=default)
        return res
    
    # ChainMap operates in reverse order so functions that appear earlier in
    # `fns` take priority.
    extras_ = dict(ChainMap(*map(_args, fns)))

    def decorator(func):
        """First get params present in func's original signature, then get
        params from additional functions which are NOT present in original 
        signature. Combine and sort param lists so positional args come first
        etc. Finally replace func's signature with our newly constructed one.
        """
        sig = inspect.signature(func)
        extras = [v for v in 
                  select(extras_, drop=sig.parameters.keys()).values()]
        parameters = sorted(
            list(sig.parameters.values()) + extras, 
            key=lambda x: (x.kind, x.default != inspect._empty)
        )
        func.__signature__ = sig.replace(parameters=parameters)
        
        @wraps(func)
        def wrapper(*args, **kwargs):
            """Execute wrapped function in a context where kwargs are 
            temporarily available as globals. Globals will be restored to 
            its prior state once execution completes.
            """
            # Order matters here: defaults must come first so user-passed 
            # args/kwargs will override them.
            kwargs = {**{p.name: p.default for p in extras},  
                      **func.__signature__.bind(*args, **kwargs).arguments}
            with temporary_global_scope(kwargs):
                return func(**kwargs)
        return wrapper
    return decorator

In [6]:
def foo(x, c, *args, a=3, e=(11, 9), b=True, f=('a', 'b', 'c'), **kwargs):
    print('foo args', args, 'foo kwargs', kwargs)
    return a*b

In [7]:
def bar(c, d=16):
    print('in bar')
    with assert_raises(Exception):
        print(f'x={x}')
    try:
        print({'c': c, 'd': d, 'a': a, 'e': e, 'b': b, 'f': f})
    except:
        print('could not find c, d, a, e, b, f')

In [8]:
getattr(bar, '__defaults__', None)

(16,)

In [9]:
getattr(bar, '__kwedefaults__', None)

In [10]:
with assert_raises(TypeError):
    bar(1, e=44)

As expected, got TypeError(bar() got an unexpected keyword argument 'e').


In [11]:
hasattr(bar, '__signature__')

False

In [12]:
@add_kwargs(foo, required=True)
def bar(q, d=16, **kwargs):
    print('bar locals', locals())
    foo(x, c, *foo_args, a=a, e=e, b=b, f=f, **foo_kwargs)
    foo_kwargs['test'] = 'new_val'
    return {'q': q, 'x': x, 'c': c, 'd': d, 'a': a, 'e': e, 'b': b, 'f': f}

In [14]:
inspect.signature(bar)

<Signature (q, x, c, d=16, foo_args=(), *, a=3, e=(11, 9), b=True, f=('a', 'b', 'c'), foo_kwargs={}, **kwargs)>

In [15]:
bar(88, 'xx', 999, a='a4', foo_kwargs={'j': 100, 'nnn': -1})

bar locals {'q': 88, 'd': 16, 'kwargs': {'x': 'xx', 'c': 999, 'foo_args': (), 'a': 'a4', 'e': (11, 9), 'b': True, 'f': ('a', 'b', 'c'), 'foo_kwargs': {'j': 100, 'nnn': -1}}}
foo args () foo kwargs {'j': 100, 'nnn': -1}


{'q': 88,
 'x': 'xx',
 'c': 999,
 'd': 16,
 'a': 'a4',
 'e': (11, 9),
 'b': True,
 'f': ('a', 'b', 'c')}

In [16]:
bar(q=98, x=1, c=2, d='dad')

bar locals {'q': 98, 'd': 'dad', 'kwargs': {'x': 1, 'c': 2, 'foo_args': (), 'a': 3, 'e': (11, 9), 'b': True, 'f': ('a', 'b', 'c'), 'foo_kwargs': {}}}
foo args () foo kwargs {}


{'q': 98,
 'x': 1,
 'c': 2,
 'd': 'dad',
 'a': 3,
 'e': (11, 9),
 'b': True,
 'f': ('a', 'b', 'c')}

In [17]:
def tri(n, z='z', x='xtri', c='ctri'):
    print('In tri')
    return n * (z+x+c)

In [18]:
# tri comes first. This means its x param should take priority over foo's. bar
# always takes first priority so the c param remains positional.
@add_kwargs(tri, foo, required=True)
def bar(c, d=16, **kwargs):
    print('bar locals:', locals())
    return {'c': c, 'n': n, 'd': d, 'x': x, 'z': z, 'a': a, 'e': e, 'b': b, 
            'f': f}

In [19]:
bar.__signature__

<Signature (c, n, d=16, x='xtri', foo_args=(), z='z', *, a=3, e=(11, 9), b=True, f=('a', 'b', 'c'), foo_kwargs={}, **kwargs)>

In [23]:
n = 100
bar(1, 11, z='zzz')
assert n == 100

bar locals: {'c': 1, 'd': 16, 'kwargs': {'x': 'xtri', 'foo_args': (), 'a': 3, 'e': (11, 9), 'b': True, 'f': ('a', 'b', 'c'), 'foo_kwargs': {}, 'n': 11, 'z': 'zzz'}}


In [24]:
bar(1, n=14, z='zzz')

bar locals: {'c': 1, 'd': 16, 'kwargs': {'x': 'xtri', 'foo_args': (), 'a': 3, 'e': (11, 9), 'b': True, 'f': ('a', 'b', 'c'), 'foo_kwargs': {}, 'n': 14, 'z': 'zzz'}}


{'c': 1,
 'n': 14,
 'd': 16,
 'x': 'xtri',
 'z': 'zzz',
 'a': 3,
 'e': (11, 9),
 'b': True,
 'f': ('a', 'b', 'c')}

In [25]:
# Foo comes first. x should be positional now.
@add_kwargs(foo, tri, required=True)
def bar(c, d=16, **kwargs):
    print('in bar')
    print('locals', locals())
    return {'c': c, 'n': n, 'd': d, 'x': x, 'z': z, 'a': a, 'e': e, 'b': b, 
            'f': f}

In [26]:
bar.__signature__

<Signature (c, n, x, d=16, z='z', foo_args=(), *, a=3, e=(11, 9), b=True, f=('a', 'b', 'c'), foo_kwargs={}, **kwargs)>

In [27]:
n = 100

In [28]:
bar(1, 11, 222, z='zzz')

in bar
locals {'c': 1, 'd': 16, 'kwargs': {'n': 11, 'z': 'zzz', 'x': 222, 'foo_args': (), 'a': 3, 'e': (11, 9), 'b': True, 'f': ('a', 'b', 'c'), 'foo_kwargs': {}}}


{'c': 1,
 'n': 11,
 'd': 16,
 'x': 222,
 'z': 'zzz',
 'a': 3,
 'e': (11, 9),
 'b': True,
 'f': ('a', 'b', 'c')}

In [29]:
bar(1, n=14, x=900, z='zzz')

in bar
locals {'c': 1, 'd': 16, 'kwargs': {'n': 14, 'z': 'zzz', 'x': 900, 'foo_args': (), 'a': 3, 'e': (11, 9), 'b': True, 'f': ('a', 'b', 'c'), 'foo_kwargs': {}}}


{'c': 1,
 'n': 14,
 'd': 16,
 'x': 900,
 'z': 'zzz',
 'a': 3,
 'e': (11, 9),
 'b': True,
 'f': ('a', 'b', 'c')}

In [30]:
n

100

In [39]:
# Tri comes first but positional=False so no n. x should have defaults. c does
# not because it's already in bar as positional.
@add_kwargs(tri, foo, required=False)
def bar(c, d=16, **kwargs):
    print('bar locals', locals())
    return {'c': c, 'n': n, 'd': d, 'x': x, 'z': z, 'a': a, 'e': e, 'b': b, 
            'f': f}

In [40]:
bar.__signature__

<Signature (c, d=16, foo_args=(), z='z', x='xtri', *, a=3, e=(11, 9), b=True, f=('a', 'b', 'c'), foo_kwargs={}, **kwargs)>

In [34]:
bar(1, 11, z='zzz')

bar locals {'c': 1, 'd': 11, 'kwargs': {'foo_args': (), 'a': 3, 'e': (11, 9), 'b': True, 'f': ('a', 'b', 'c'), 'foo_kwargs': {}, 'z': 'zzz', 'x': 'xtri'}}


{'c': 1,
 'n': 100,
 'd': 11,
 'x': 'xtri',
 'z': 'zzz',
 'a': 3,
 'e': (11, 9),
 'b': True,
 'f': ('a', 'b', 'c')}

In [35]:
lmap(inspect.signature, tri, foo)

[<Signature (n, z='z', x='xtri', c='ctri')>,
 <Signature (x, c, *args, a=3, e=(11, 9), b=True, f=('a', 'b', 'c'), **kwargs)>]

In [38]:
lmap(lambda x: {k: v.kind for 
                k, v in inspect.signature(x).parameters.items()}, tri, foo)

[{'n': <_ParameterKind.POSITIONAL_OR_KEYWORD: 1>,
  'z': <_ParameterKind.POSITIONAL_OR_KEYWORD: 1>,
  'x': <_ParameterKind.POSITIONAL_OR_KEYWORD: 1>,
  'c': <_ParameterKind.POSITIONAL_OR_KEYWORD: 1>},
 {'x': <_ParameterKind.POSITIONAL_OR_KEYWORD: 1>,
  'c': <_ParameterKind.POSITIONAL_OR_KEYWORD: 1>,
  'args': <_ParameterKind.VAR_POSITIONAL: 2>,
  'a': <_ParameterKind.KEYWORD_ONLY: 3>,
  'e': <_ParameterKind.KEYWORD_ONLY: 3>,
  'b': <_ParameterKind.KEYWORD_ONLY: 3>,
  'f': <_ParameterKind.KEYWORD_ONLY: 3>,
  'kwargs': <_ParameterKind.VAR_KEYWORD: 4>}]

In [36]:
# Foo comes first but pos=False, so tri still overrides and we get x w/
# default val.
@add_kwargs(foo, tri, required=False)
def bar(c, d=16, **kwargs):
    print('in bar')
    print('locals', locals())
    return {'c': c, 'n': n, 'd': d, 'x': x, 'z': z, 'a': a, 'e': e, 'b': b, 
            'f': f}

In [37]:
bar.__signature__

<Signature (c, d=16, z='z', x='xtri', foo_args=(), *, a=3, e=(11, 9), b=True, f=('a', 'b', 'c'), foo_kwargs={}, **kwargs)>

In [1126]:
bar(1, 11, z='zzz')

in bar
locals {'c': 1, 'd': 11, 'kwargs': {'z': 'zzz', 'x': 'xtri', 'foo_args': (), 'a': 3, 'e': (11, 9), 'b': True, 'f': ('a', 'b', 'c'), 'foo_kwargs': {}}}


{'c': 1,
 'n': 100,
 'd': 11,
 'x': 'xtri',
 'z': 'zzz',
 'a': 3,
 'e': (11, 9),
 'b': True,
 'f': ('a', 'b', 'c')}

## More involved approach: construct a new function rather than messing with signature

In [1088]:
import ast
from ast import FunctionDef, arguments, Return, Module
from inspect import getsource
import re
from types import FunctionType

In [662]:
bar_txt = """
def bar(c, d=16):
    def _inner_(test):
        return test
    print('in bar')
    with assert_raises(Exception):
        print(f'x={x}')
    try:
        print({'c': c, 'd': d, 'a': a, 'e': e, 'b': b, 'f': f})
    except:
        print('could not find c, d, a, e, b, f')
"""

In [663]:
@add_kwargs(tri, foo, positional=True)
def bar(c, d=16):
    print('in bar')
    tri_res = tri(n, z, x, c)
    return tri_res, c*d

old {'x': <Parameter "x='xtri'">, 'c': <Parameter "c='ctri'">, 'a': <Parameter "a=3">, 'e': <Parameter "e=(11, 9)">, 'b': <Parameter "b=True">, 'f': <Parameter "f=('a', 'b', 'c')">, 'n': <Parameter "n">, 'z': <Parameter "z='z'">}
ex [<Parameter "x='xtri'">, <Parameter "a=3">, <Parameter "e=(11, 9)">, <Parameter "b=True">, <Parameter "f=('a', 'b', 'c')">, <Parameter "n">, <Parameter "z='z'">]
params [<Parameter "c">, <Parameter "n">, <Parameter "d=16">, <Parameter "x='xtri'">, <Parameter "z='z'">, <Parameter "a=3">, <Parameter "e=(11, 9)">, <Parameter "b=True">, <Parameter "f=('a', 'b', 'c')">]


In [664]:
def func_str_from_signature(func, func_str=None):
    func_str = func_str or getsource(func)
    old_sig = re.search('def [^(]+(\(.*\)):', func_str).group(1)
    return func_str.replace(old_sig, str(func.__signature__))

<>:3: DeprecationWarning: invalid escape sequence \(
<>:3: DeprecationWarning: invalid escape sequence \(
<>:3: DeprecationWarning: invalid escape sequence \(
<ipython-input-664-332bcc1012a5>:3: DeprecationWarning: invalid escape sequence \(
  old_sig = re.search('def [^(]+(\(.*\)):', func_str).group(1)


In [665]:
print(getsource(bar))

@add_kwargs(tri, foo, positional=True)
def bar(c, d=16):
    print('in bar')
    tri_res = tri(n, z, x, c)
    return tri_res, c*d



In [666]:
bar.__signature__

<Signature (c, n, d=16, x='xtri', z='z', *, a=3, e=(11, 9), b=True, f=('a', 'b', 'c'))>

In [667]:
print(match_signature(bar))

NameError: name 'match_signature' is not defined

In [668]:
def function_from_string(func_str):
    name = re.search('def ([^(]+)\(.*\):', func_str).group(1)
    print('name', name)
    print('pre tree')
    tree = ast.parse(func_str, mode='exec')
    print('pre code')
    code = compile(tree, filename='<fname>', mode='exec')
    print('pre exec')
    exec(code, globals())
    print('pre return')
    return globals()[name]

<>:2: DeprecationWarning: invalid escape sequence \(
<>:2: DeprecationWarning: invalid escape sequence \(
<>:2: DeprecationWarning: invalid escape sequence \(
<ipython-input-668-dae797d36328>:2: DeprecationWarning: invalid escape sequence \(
  name = re.search('def ([^(]+)\(.*\):', func_str).group(1)


In [669]:
four_txt = """
def four(c, d=16):
    def _inner_(test):
        return test
    print('in four')
    return c*d
"""

In [670]:
newfunc = function_from_string(four_txt)

name four
pre tree
pre code
pre exec
pre return


In [671]:
newfunc(3)

in four


48

In [672]:
newfunc.__name__

'four'

In [673]:
four(2)

in four


32

In [674]:
@add_kwargs(tri, foo, positional=True)
def bar(c, d=16):
    print('in bar')
    tri_res = tri(n, z, x, c)
    return tri_res, c*d

old {'x': <Parameter "x='xtri'">, 'c': <Parameter "c='ctri'">, 'a': <Parameter "a=3">, 'e': <Parameter "e=(11, 9)">, 'b': <Parameter "b=True">, 'f': <Parameter "f=('a', 'b', 'c')">, 'n': <Parameter "n">, 'z': <Parameter "z='z'">}
ex [<Parameter "x='xtri'">, <Parameter "a=3">, <Parameter "e=(11, 9)">, <Parameter "b=True">, <Parameter "f=('a', 'b', 'c')">, <Parameter "n">, <Parameter "z='z'">]
params [<Parameter "c">, <Parameter "n">, <Parameter "d=16">, <Parameter "x='xtri'">, <Parameter "z='z'">, <Parameter "a=3">, <Parameter "e=(11, 9)">, <Parameter "b=True">, <Parameter "f=('a', 'b', 'c')">]


In [675]:
bar

<function __main__.bar(c, n, d=16, x='xtri', z='z', *, a=3, e=(11, 9), b=True, f=('a', 'b', 'c'))>

In [676]:
bar_new = function_from_string(match_signature(bar))

NameError: name 'match_signature' is not defined

In [677]:
bar_new.__signature__

NameError: name 'bar_new' is not defined

In [678]:
bar_new.__module__

NameError: name 'bar_new' is not defined

In [679]:
bar_new('c4', 3, z='z11')

NameError: name 'bar_new' is not defined

### Issue

`function_from_string` doesn't seem to work inside decorator. The eval line oddly calls `add_kwargs` again. On this second execution, inspect is unable to retrieve the source file for func.

In [680]:
def add_kwargs(*fns, positional=False):
    """When multiple functions contain a parameter with the same name, 
    priority is determined by the order of `fns`.
    """
    param_types = [Parameter.POSITIONAL_OR_KEYWORD, 
                   Parameter.POSITIONAL_ONLY,
                   Parameter.KEYWORD_ONLY]
    if not positional: param_types = param_types[-1:]
    def _args(fn): 
        return {k: v for k, v in params(fn).items() if v.kind in param_types}
    old_args = dict(ChainMap(*map(_args, fns)))
    print('ADD_KWARGS')

    def decorator(func):
        sig = inspect.signature(func)
        extras = [Parameter(k, v.kind, default=v.default) for k, v in 
                  select(old_args, drop=sig.parameters.keys()).items()]
        parameters = sorted(
            list(sig.parameters.values()) + extras, 
            key=lambda x: (x.kind, x.default != inspect._empty)
        )
        func.__signature__ = sig.replace(parameters=parameters)
        print('DECO 0')
        src = getsource(func)
        print('DECO 1')
        _func = func_str_from_signature(func, src)
        print('DECO 2')
        _func = function_from_string(_func)
        print('DECO 3')
        
        @wraps(func)
        def wrapper(*args, **kwargs):
            print('CALL')
            return func(*args, **kwargs)
        return wrapper
    return decorator

In [681]:
@add_kwargs(tri, foo, positional=True)
def bar(c, d=16):
    print('in bar')
    tri_res = tri(n, z, x, c)
    return tri_res, c*d

ADD_KWARGS
DECO 0
DECO 1
DECO 2
name bar
pre tree
pre code
pre exec
ADD_KWARGS
DECO 0


OSError: could not get source code

In [682]:
getsource(bar)

"@add_kwargs(tri, foo, positional=True)\ndef bar(c, d=16):\n    print('in bar')\n    tri_res = tri(n, z, x, c)\n    return tri_res, c*d\n"

In [683]:
bar(2)

TypeError: missing a required argument: 'n'